<a href="https://colab.research.google.com/github/syeong1218/RL/blob/master/Ch4_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ch4. A2C

REINFORCE 알고리즘 단점 : 에피소드가 끝날 때까지 기다려야 한다, 래디언트 분산이 매우 크다. -> 개선한 알고리즘 : A2C

- 목적함수의 그래디언트 변형

기존 :
$$=\sum _{t=0} ^T \{E_{\tau \sim p_\theta}[\gamma^t \nabla_\theta \log  \pi_\theta(u_t|x_t)(\sum _{k=t} ^T \gamma^{k-t}r(x_k, u_k))]\}$$
<br/>
마르코스 가정과 베이즈 정리 이용해서 정리하면,
$$\nabla_\theta J(\theta)=\sum_{t=0}^T(E_{x_t\sim p_\theta(x_t), u_t\sim \pi_\theta(u_t\mid x_t)}[\nabla_\theta log\pi_\theta(u_t \mid x_t)Q^{\pi_\theta}(x_t, u_t)])$$
<br/>
=> 기존의 반환값대신 행동가치를 이용하여 계산하면 된다. 

즉, 에피소드가 끝날 때까지 기다릴 필요가 없다.

- 분산 줄이는 방법

목적함수 그래디언트 식에서 행동가치 함수 대신, 파라미터 $b_t$를 대입한다.

여기서 $b_t$는 상수이거나 행동 $u_t$의 함수가 아니고 베이스라인이라고 한다. 목적함수 그래디언트의 분산을 최소화하는 값이 최적값이다. 일반적으로 상태가치 함수를 사용한다.

<br/>

따라서,
$$ \nabla_\theta J(\theta)=0$$
이 된다.

<br/>

즉, 목적함수 그래디언트 식의 행동가치함수에서 $b_t$를 빼도 기댓값은 변하지 않는다.
<br/>
$$\nabla_\theta J(\theta)=\sum _{t=0} ^T (E_{x_t \sim d_\theta (x_t),u_t \sim \pi_\theta(u_t|x_t)}[\nabla_\theta \log \pi_\theta(u_t|x_t)(Q^{\pi_\theta}(x_t, u_t)-b_t)])$$
$$=\sum _{t=0} ^T (E_{x_t \sim d_\theta (x_t),u_t \sim \pi_\theta(u_t|x_t)}[\nabla_\theta \log \pi_\theta(u_t|x_t)(Q^{\pi_\theta}(x_t, u_t)-V^{\pi_\theta}(x_t))])$$
$$=\sum _{t=0} ^T (E_{x_t \sim d_\theta (x_t),u_t \sim \pi_\theta(u_t|x_t)}[\nabla_\theta \log \pi_\theta(u_t|x_t)(A^{\pi_\theta}(x_t, u_t))])$$

<br/>

$A^{\pi_\theta}(x_t, u_t)$를 <b>어드밴티지 함수</b>이다. 상태변수는 모든 행동에 대한 행동가치의 평균값으로, 상태변수 $x_t$에서 선택된 행동 $u_t$가 평균에 비해 얼마나 좋은지를 평가하는 척도로 해석 가능하다.

<br/>

=> 목적함수 그래디언트는 어드밴티지 함수에 비례한다.


행동가치 함수 식을 상태가치 함수식으로 나타내어 어드밴티지 함수를 다시 쓰면

<br/>

$$A^{\pi_\theta}(x_t, u_t))=Q^{\pi_\theta}(x_t, u_t)-V^{\pi_\theta}(x_t)=r(x_t,u_t)+\gamma V^{\pi_\theta}(x_{t+1})-V^{\pi_\theta}(x_t) $$

=> 어드밴티지 계산 문제는 이제 상태가치 계산 문제로 바뀌었다. $\pi_\theta(u_t\mid x_t)$는 정책 신경망을, $\phi$는 가치신경망을 추정하여 상태가치 함수를 근사한다.

<br/>

![대체 텍스트](https://github.com/syeong1218/RL/blob/master/4-3.jpg?raw=true)

정책 신경망 : 에이전트가 어떻게 행동해야 하는지 알려주는 **액터 신경망**

가치 신경망 : 그 행동을 평가하는 **크리틱 신경망**

두 신경망은 중첩되는 부분이 많아 다음과 같이 쓰인다.

<br/>

![대체 텍스트](https://github.com/syeong1218/RL/blob/master/4-4.jpg?raw=true)

<br/>

가치함수를 근사하는 함수를 구하기 위해 벨만 방정식을 이용하면 아래와 같다.

<br/>

$$V_{\phi }(x_{t})\approx r(x_{t },u_{t } )+\gamma V_{\phi}(x_{t+1})$$

<br/>

시간차 타깃을 $y_{i}=r(x_{i},u_{i})+\gamma V_{\phi }(x_{i+1})$로 설정하면 손실함수가 최소가 되도록 근사 가치함수를 추정할 수 있다.

<br/>

크리틱 신경망의 손실함수는 아래와 같다.

<br/>

$$Loss_{critic}(\phi )=\frac{1}{2}\sum_{i}^{}\left \| r(x_{i},u_{i})+\gamma V_{\phi }(x_{i+1})-V_{\phi }(x_{i}) \right \|^{2}$$

<br/>

어드밴티지 함수도 아래와 같이 근사할 수 있다.

<br/>

$$A_{\phi}(x_{i},u_{i})\approx r(x_{i},u_{i})+\gamma V_{\phi }(x_{i+1})-V_{\phi }(x_{i})$$

<br/>

액터 신경망의 손실함수는 아래와 같다.

<br/>

$$Loss_{actor}(\theta )\approx -\sum_{i}^{}(log\pi _{\theta }(u_{i}|x_{i})A_{\phi }(x_{i},u_{i}))$$

<br/>

최적 정책을 찾는 파라미터는 아래와 같다.

<br/>

$$\theta \leftarrow \theta -\alpha \triangledown _{\theta }\sum_{i}^{}(log\pi _{\theta }(u_{i}|x_{i})A_{\phi }(x_{i},u_{i}))$$

<br/>

마이너스 부호 : 손실함수를 최소화하도록 파라미터가 엡데이트되고, 목적함수는 최대로 해야하기 때문이다.

<br/>

A2C의 알고리즘은 아래 그림과 같다.

![대체 텍스트](https://github.com/syeong1218/RL/blob/master/4-5.jpg?raw=true)

![대체 텍스트](https://github.com/syeong1218/RL/blob/master/4-6.jpg?raw=true)


**N개의 샘플->정책 업데이트->반복**

  1. 크리틱과 액터 신경망의 파라미터 $\phi,\theta$ 초기화
  2.  반복
    1. N시간스텝 동안 아래의 과정 반복
       1. 정책으로 행동을 확률적으로 선택
       2. 행동의 보상과 다음 상태변수를 측정
       3. 샘플 저장
    2. 시간차 타깃 계산
    3. 크리틱 신경망의 손실함수 계산 
    4. 어드밴티지 계산
    5. 크리틱 신경망 업데이트
    6. 액터 신경망 업데이트

<br/>

행동이 **이산공간**이면, 액터 신경망 출력층의 뉴런의 개수는 **유한개**이다.

<br/>

행동이 **연속공간**이면, 액터 신경망 출력층의 뉴런의 개수는 **무한개**이다.

<br/>

![대체 텍스트](https://github.com/syeong1218/RL/blob/master/4-8.jpg?raw=true)

=> 액터 신경망이 각각의 행동변수가 갖는 가우시안 분포의 평균값과 분산을 출력하도록 한다. 그러면 액터 신경망 출력층의 뉴런 개수는 행동변수의 차원당 2개면 된다.


